In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [2]:
eng.quit()

In [1]:
import json
from febio_python.feb import FEBio_feb
import numpy as np

In [2]:
with open("./test.json", "r") as jsonfile:
  json_data = json.load(jsonfile)
json_data.keys()

dict_keys(['NODES', 'ELEMENTS', 'NODESETS', 'ELEMTSETS', 'SURFACES', 'VIRTUAL_NODES'])

In [3]:
feb = FEBio_feb("./TEMPLATE.feb")

In [4]:
feb.replace_nodes(json_data["NODES"])

C:\Users\igornobrega\AppData\Local\Programs\Python\Python39\lib\site-packages\febio_python\feb\FEBio_feb.py:169: UserWarning: Replacing nodes data with new amound of nodes.                    This can lead to malfunction in feb file exectuion
  warnings.warn("Replacing nodes data with new amound of nodes.\


In [5]:
tet4elems = np.array(json_data["ELEMENTS"]["TETRA"]) + 1
tet4elems

array([[    1,     2,     3,     4],
       [    5,     6,     7,     8],
       [    9,    10,    11,    12],
       ...,
       [72360, 72359, 50645, 72476],
       [50645, 50646, 72359, 50643],
       [50645, 72359, 50646, 72476]])

In [6]:
feb.add_elements({"LV": tet4elems}, 1, eltype="tet4")

In [7]:
import xml.etree.ElementTree as ET

In [8]:
initial_el_id = 1
for (name, surf) in json_data["SURFACES"].items():
    el_root = ET.Element("Surface")
    el_root.set("name", name)

    for i, xyz in enumerate(surf):
        subel = ET.SubElement(el_root, "tri3")
        subel.set("id", str(i + initial_el_id))
        subel.text = ",".join(map(str, xyz))
    initial_el_id = i + 1

    feb.Geometry.extend([el_root])